In [25]:
import pandas as pd
import mysql.connector
import sqlalchemy as sa
import getpass

host = 'localhost'
user = 'root'
password = getpass.getpass()
database = 'goodreads'

def getconn():
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    return conn

pool = sa.create_engine(
    "mysql+mysqlconnector://",
    creator=getconn,
)

with pool.connect() as db_conn:
    results = db_conn.execute(sa.text("SELECT NOW()")).fetchone()
    print("Current time: ", results[0])

Current time:  2023-05-23 13:47:45


In [26]:
to_read_query = sa.text(
    "SELECT * FROM to_read;"
)

id_dict_query = sa.text(
    "SELECT * FROM id_dict;"
)

to_read_df = pd.read_sql_query(to_read_query, con=pool.connect())
id_dict_df = pd.read_sql_query(id_dict_query, con=pool.connect())

display(to_read_df.head())
display(id_dict_df.head())

,book_id,user_id
0,8,9
1,398,15
2,275,15
3,7173,37
4,380,34


,book_id,goodreads_book_id
0,1,2767052
1,2,3
2,3,41865
3,4,2657
4,5,4671


In [34]:
id_dict_df.book_id = id_dict_df.book_id.astype('str')
id_dict_df.goodreads_book_id = id_dict_df.goodreads_book_id.astype('str')
id_dict_df.dtypes

book_id              object
goodreads_book_id    object
dtype: object

In [35]:
id_dict = dict(zip(id_dict_df.book_id, id_dict_df.goodreads_book_id))
print(id_dict)

{'1': '2767052', '2': '3', '3': '41865', '4': '2657', '5': '4671', '6': '11870085', '7': '5907', '8': '5107', '9': '960', '10': '1885', '11': '77203', '12': '13335037', '13': '5470', '14': '7613', '15': '48855', '16': '2429135', '17': '6148028', '18': '5', '19': '34', '20': '7260188', '21': '2', '22': '12232938', '23': '15881', '24': '6', '25': '136251', '26': '968', '27': '1', '28': '7624', '29': '18135', '30': '8442457', '31': '4667024', '32': '890', '33': '930', '34': '10818853', '35': '865', '36': '3636', '37': '100915', '38': '14050', '39': '13496', '40': '19501', '41': '28187', '42': '1934', '43': '10210', '44': '15931', '45': '4214', '46': '43641', '47': '19063', '48': '4381', '49': '49041', '50': '30119', '51': '256683', '52': '428263', '53': '113436', '54': '11', '55': '5129', '56': '1162543', '57': '37435', '58': '2956', '59': '24178', '60': '1618', '61': '22557272', '62': '119322', '63': '6185', '64': '10917', '65': '4981', '66': '18405', '67': '128029', '68': '22628', '69':

In [36]:
sample = to_read_df.head(5).copy()
display(sample)
sample['book_id'] = sample['book_id'].replace(id_dict)
display(sample)

,book_id,user_id
0,8,9
1,398,15
2,275,15
3,7173,37
4,380,34


,book_id,user_id
0,5107,9
1,1097,15
2,49750,15
3,102030,37
4,11297,34


In [37]:
to_read_df.book_id.replace(id_dict, inplace=True)
to_read_df.head()

,book_id,user_id
0,5107,9
1,1097,15
2,49750,15
3,102030,37
4,11297,34


In [38]:
to_read_df.rename(columns={'book_id': 'goodreads_book_id'}, inplace=True)
to_read_df.columns

Index(['goodreads_book_id', 'user_id'], dtype='object')

In [39]:
to_read_df.to_sql(name='new_to_read', con=pool, if_exists='append', index=False)

912705